In [143]:
# To read the PDF
from pypdf import PdfReader
#to read environment variables, list PDFs in folder etc
import os
from dotenv import load_dotenv

# to clean text from PDFs
import string

#to deal with embeddings
import hdbscan
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains.summarize import load_summarize_chain
from langchain_community.embeddings import GPT4AllEmbeddings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [100]:
%load_ext dotenv
%dotenv

In [102]:
load_dotenv()

True

In [68]:
#specific file
pdf_file ="../raw_data/pdf_reports/2016/160602 (Marex).pdf"
reader = PdfReader(pdf_file)
page = reader.pages[0]
text =page.extract_text()
#comment = text.partition("COT")[2]
comment = text
comment = comment.lower()
comment = ''.join(char for char in comment if not char.isdigit())
for punctuation in string.punctuation:
    comment = comment.replace(punctuation, '') 
comment = comment.replace('\n'," ")
comment = comment.split("         ")[-1]
len(comment)

16

In [42]:
comment.split("         ")[-1]

'to say the least it was a violent week for cocoa with prices netting losses of £ respectively yet still rallying nearly points in each market off the lows charted yesterday leaving the recover y almost as equally impressive as the decline  as discussed a shoddy macro backdrop steamrolling of the  equity and commodity markets solid arrivals and perhaps an element of compulsory hedging accelerated the path lower taking prices deep into oversold territory by midweek  value based buying short coveringand a positive chart signal helped catapult prices off the lows with a weaker trending sterling providing the  additional boost for the london market  regarding short covering one other curious aspect of trading this  week involved open interest which remained curiously stable throughout the decline with any material offsets of positions only registered after the market rebounded yesterday  volumes by themselves were  another key feature with each platform averaging about  contracts a day  in

In [70]:
#listing the files from a specific folder
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("../raw_data/pdf_reports/2016") if isfile(join("../raw_data/pdf_reports/2016", f))]
onlyfiles

['161010.pdf',
 '160108.pdf',
 '160907.pdf',
 '160205.pdf',
 '160718.pdf',
 '160104.pdf',
 '161122.pdf',
 '160413.pdf',
 '161025.pdf',
 '161228.pdf',
 '161229.pdf',
 '161018.pdf',
 '161219.pdf',
 '160524.pdf',
 '160302.pdf',
 '160111.pdf',
 '160316.pdf',
 '161111.pdf',
 '160210.pdf',
 '160407.pdf',
 '161206.pdf',
 '160706.pdf',
 '160425.pdf',
 '160411.pdf',
 '160816.pdf',
 '160226.pdf',
 '160506.pdf',
 '160412.pdf',
 '160930.pdf',
 '160509.pdf',
 '160314.pdf',
 '160818.pdf',
 '160727.pdf',
 '161213.pdf',
 '161006.pdf',
 '161220.pdf',
 '160408.pdf',
 '160323.pdf',
 '160712.pdf',
 '160830.pdf',
 '161121.pdf',
 '160118.pdf',
 '160726.pdf',
 '160308.pdf',
 '161125.pdf',
 '161114.pdf',
 '160629.pdf',
 '161205.pdf',
 '160307.pdf',
 '160502.pdf',
 '160121.pdf',
 '160128.pdf',
 '160419.pdf',
 '160122.pdf',
 '160915.pdf',
 '161108.pdf',
 '161017.pdf',
 '160610.pdf',
 '160405.pdf',
 '160901.pdf',
 '160811.pdf',
 '160418.pdf',
 '160303.pdf',
 '160527.pdf',
 '160217.pdf',
 '160224.pdf',
 '160929.p

In [ ]:
#look of PDF text before cleaning
files = "160108.pdf"
text = extract_text(f"../raw_data/pdf_reports/2016/{files}", maxpages = 1)
text

In [74]:
%%time
comments_db = []
files_not_processed =[]
for files in onlyfiles:
    #text = extract_text(f"../raw_data/pdf_reports/2016/{files}", maxpages = 1)
    reader = PdfReader(f"../raw_data/pdf_reports/2016/{files}")
    page = reader.pages[0]
    text =page.extract_text()
    comment = text.partition("COT")[2]
    comment = comment.lower()
    comment = ''.join(char for char in comment if not char.isdigit())
    for punctuation in string.punctuation:
       comment = comment.replace(punctuation, '') 
    comment = comment.replace('\n'," ")
    comment = comment.split("         ")[-1]
    if len(comment) < 25:
        files_not_processed.append(files)
    else:
        comments_db.append(comment)
comments_db

CPU times: user 35.2 s, sys: 38.3 ms, total: 35.3 s
Wall time: 35.3 s


[' trade at settlement     open interest      new  york  spreads      monday october     cocoa market report  open interest    oi nr  com     index      net com   net ls com  delaware       in london a heavy bout of system related sell ing and discretionary fund liquidation were the mai n culprits in driving the ma rket  down  basis the front month  system related selling appeared from the onset eventually  penetrating  basis mar the   day moving average electing multiple sell stop s in the process a selling vacuum ensued as spec l iquidation added a furt her leg  lower with market finally finding support at  b asis mar  trade related buyers prevented the market from brea king through   basis mar the recent low where many potent ial sell stops could be lurking for the duration of  the trading session n earby  structure finally showed signs of weakness during the flat price sell off as its prem ium quickly deteriorated with decmar  closing at  premium and marmay moving into c ontango se

In [75]:
len(comments_db)

241

In [57]:
comments_db[49]

'joseph p mallaney   todd c lunstead michael dann jonathan proctor andrew proctor michael juliano devon dinneen jamie ball graham phillipsdateldn volumes ny volumes      london  spreads      suprt  res ice tas volumes  marnet chngfeb     may new york  mar   valid certs       albany hampton roads  baltimore totalkey rates       mng  netnet ls oi    trade at settlement    open interest      new  york  spreads      thursday march   cocoa market report open interest   oi nr  com   index  net com   net ls com delaware      after primarily trading back and forth over the last two weeks albeit with an upward bias prices in london built  on yesterday’s momentum to settle at a fresh two month high  some extreme moves in the foreign exchange markets might have provided the additional boost as sterling ultimately fell more than  against the euro following additional stimulus measures by the ecb  while the euro strength subsequent a  basis point rate cut to  negative  and an expanded asset purchas

In [76]:
files_not_processed

['160104.pdf',
 '160203.pdf',
 '160329.pdf',
 '160204.pdf',
 '160825.pdf',
 '160331.pdf',
 '160602 (Marex).pdf']

In [121]:
comments_db_df = pd.DataFrame(comments_db)
comments_db_df.to_csv(path_or_buf="../raw_data/pdf_reports/comments_2016.csv",index=False)

In [ ]:
#embedding

In [108]:
gpt4all_embd = GPT4AllEmbeddings()

100%|█████████████████████████████████████████████████████████████████████████| 45.9M/45.9M [00:07<00:00, 6.36MiB/s]


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [124]:
%%time
#to embed a single piece of text
query_result = gpt4all_embd.embed_query(comments_db[0])

CPU times: user 2.94 s, sys: 0 ns, total: 2.94 s
Wall time: 738 ms


In [129]:
%%time
#to embed multiple pieces of text
doc_result = gpt4all_embd.embed_documents(comments_db)
doc_result

CPU times: user 9min 55s, sys: 2.92 s, total: 9min 58s
Wall time: 2min 30s


[[-0.05933646857738495,
  -0.06845249980688095,
  0.060500726103782654,
  0.023023096844553947,
  0.011749990284442902,
  0.03674953803420067,
  -0.026445966213941574,
  0.13157004117965698,
  0.03263464570045471,
  0.03583456948399544,
  -0.00702622439712286,
  0.05678967759013176,
  -0.03689942881464958,
  -0.06054643541574478,
  0.03334863856434822,
  -0.032760363072156906,
  -0.033833298832178116,
  -0.0012871926883235574,
  -0.09236519038677216,
  0.0887591689825058,
  -0.0764479786157608,
  -0.017751824110746384,
  -0.1309986710548401,
  0.009433956816792488,
  0.023092400282621384,
  -0.0027503385208547115,
  -0.028193989768624306,
  -0.0024593123234808445,
  0.024033697322010994,
  -0.055688127875328064,
  -0.030811265110969543,
  0.08060692250728607,
  0.01010993029922247,
  0.03330593928694725,
  -0.010962852276861668,
  -0.006827082019299269,
  0.015322543680667877,
  -0.013236252591013908,
  0.04800141975283623,
  -0.039288479834795,
  0.013792610727250576,
  -0.03651661425

In [132]:
len(doc_result)

241

In [134]:
hdb = hdbscan.HDBSCAN(min_samples=3, min_cluster_size=3).fit(doc_result)
hdb.labels_

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  0,  0,  0, -1, -1, -1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  0, -1, -1,  0, -1,  1, -1,  0, -1, -1, -1, -1, -1,
       -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1,  1,  0, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1,
       -1, -1, -1, -1, -1

In [135]:
hdb.outlier_scores_

array([0.12377707, 0.11296396, 0.0412268 , 0.00431169, 0.1312749 ,
       0.32891157, 0.00927263, 0.3332661 , 0.05556474, 0.09642357,
       0.29447528, 0.27625234, 0.26099976, 0.16977193, 0.32913341,
       0.00865932, 0.29647474, 0.29912884, 0.25051276, 0.31677281,
       0.14363675, 0.09744723, 0.15778816, 0.04690834, 0.03407516,
       0.06260645, 0.0471761 , 0.1784305 , 0.21448001, 0.26445818,
       0.27582476, 0.03462653, 0.27315884, 0.41941459, 0.        ,
       0.02287688, 0.12291867, 0.08568888, 0.29170737, 0.30081635,
       0.11858978, 0.15316798, 0.32765181, 0.22232283, 0.18169023,
       0.        , 0.26741131, 0.06321568, 0.08327259, 0.21457622,
       0.1385791 , 0.14414101, 0.29500154, 0.0159716 , 0.03553177,
       0.15945385, 0.13402494, 0.04156014, 0.52516747, 0.16608708,
       0.39228801, 0.09859296, 0.05413311, 0.24924937, 0.22468491,
       0.        , 0.07927674, 0.08313156, 0.1350486 , 0.06015834,
       0.13316906, 0.31745659, 0.11475   , 0.07226672, 0.12145

In [145]:
threshold = pd.Series(hdb.outlier_scores_).quantile(0.9)
outliers = np.where(hdb.outlier_scores_ > threshold)[0]
plt.scatter(doc_result, s=50, linewidth=0, c='gray', alpha=0.25)
plt.scatter(doc_result[outliers], s=50, linewidth=0, c='red', alpha=0.5)

TypeError: scatter() missing 1 required positional argument: 'y'

In [137]:
%%time
from gensim.models import Word2Vec
word2vec = Word2Vec(sentences=comments_db)
wv = word2vec.wv

CPU times: user 118 ms, sys: 27.1 ms, total: 145 ms
Wall time: 115 ms


In [138]:
wv.key_to_index

{' ': 0,
 'e': 1,
 't': 2,
 'n': 3,
 'o': 4,
 'a': 5,
 'i': 6,
 'r': 7,
 's': 8,
 'l': 9,
 'd': 10,
 'h': 11,
 'c': 12,
 'p': 13,
 'm': 14,
 'u': 15,
 'g': 16,
 'f': 17,
 'y': 18,
 'w': 19,
 'v': 20,
 'b': 21,
 'k': 22,
 'x': 23,
 'j': 24,
 'q': 25,
 '’': 26,
 '£': 27,
 '\x0c': 28,
 'z': 29,
 '–': 30}

In [ ]:
#below are trials to extract text from the PDFs

In [66]:
text = extract_text('raw_data/pdf_reports/210113.pdf', maxpages = 1)
text

'COCOA MARKET REPORT\n\n   Arb Settle Basis Ldn Close on 20-12-24\n\nWednesday, January 13, 2021\n\nSource:  Bloomberg and the I.C.E.\n\nI.C.E EU\nCLOSE\n1703\n1648\n1631\n1619\n1626\n1623\n1625\n1627\n1629\n1636\n\nMAR 21\nMAY 21\nJUL 21\nSEP 21\nDEC 21\nMAR 22\nMAY 22\nJUL 22\nSEP 22\nDEC 22\nLondon Due: \n\n9\n7\n3\n0\n1\n1\n0\n-1\n-1\n-1\nMAY 21\n\nHI\n1,721\n1,663\n1,644\n1,632\n1,637\n1,634\n1,635\n1,636\n\nLOW\n1,683\n1,633\n1,620\n1,614\n1,619\n1,618\n1,624\n1,630\n\n-£8\n\nLower\n\nVOLUME\n5252\n5058\n3610\n1340\n862\n170\n90\n50\n\nOPEN INTEREST \n248\n-734\n92\n376\n-99\n67\n-9\n22\n\n73,478\n39,523\n29,528\n27,699\n34,104\n10,766\n4,240\n2,074\n406\n18\n221,836\n\n4\n-33\n\n16,432\nStock Market Indices\n31060.47\nDJIA\n3809.84\nS&P500\nNASDAQ\n13128.95\nFTSE 100 IDX  6745.52\nNIKKEI225\n28456.6\nCommodity Indices:\n471.72\nCCI\n\nBCOM\n\nGSCI\n\nTRJ CRB\n\nBltc Frt Idx\n\n80.61\n\n432.40\n\n175.19\n\n1849\n\n90.343\n\nDXY\nCommodities (2nd Pos.):\nSugar\n\n14.95\n\nLDN VOLU

In [67]:
len(text)

9739

In [99]:
comment = text.partition("COT")[2]
comment

'\nw/Opt.\n\nEU COT\nw/Opt.\n\nValid\n2,000\n\n5-Jan\n\nExp\n3,290\n\nCom +\n69,872\n\nCom +\n5-Jan 155,804\n\nSDU (10 mts)\nNon-Tend\n254\n\nLive\n\nCom -\n135,389\n\nCom -\n176,155\n\nCom Net\n-65,517\n\nCom Net\n-20,351\n\nTotal\n5,708\n\nSwp +\n22,873\n\nSwp +\n17,953\n\nValid\n20\n\nSwp -\n4,646\n\nSwp -\n17,527\n\nExp\n73\n\nSwp Net\n18,227\n\nSwp Net\n426\n\nLDU (100 mts)\nNon-Tend\n6\n\nMng $ +\n50,955\n\nMng $ +\n19,741\n\nLive\n\nMng $ -\n23,808\n\nMng $ -\n3,074\n\nTotal\n99\n\nValid\n32\nMng $ Net\n27,147\n\nMng $ Net\n16,667\n\nExp\n32\n\nOthr +\n18,893\n\nOthr +\n3,360\n\nBDU (1,000 mts)\nNon-Tend\n\nOthr -\n5,963\n\nOthr -\n884\n\nLive\n1\n\nOthr Net\n12,930\n\nOthr Net\n2,476\n\nTotal\n65\n\nNR +\n14,778\n\nNR +\n3,680\n\nValid\n54,000\n\nNR -\n7,564\n\nNR -\n2,898\n\nExp\n72,200\n\nNR Net\n7,214\n\nNR Net\n782\n\nTotal\nNon-Tend\n3,140\n\nOI\n254,195\n\nOI\n253,288\n\nLive\n1,000\n\nNet Spec\n65,518\n\nNet Spec\n20,351\n\nALL\n130,340\n\nChange\n1,854\n\nChange\n-5,765

In [104]:
comment = comment.lower()
comment = ''.join(char for char in comment if not char.isdigit())

for punctuation in string.punctuation:
   comment = comment.replace(punctuation, '') 

comment = comment.replace('\n',"")
len(comment)

4076

In [105]:
comment

'wopteu cotwoptvalidjanexpcom com jan sdu  mtsnontendlivecom com com netcom nettotalswp swp validswp swp expswp netswp netldu  mtsnontendmng  mng  livemng  mng  totalvalidmng  netmng  netexpothr othr bdu  mtsnontendothr othr liveothr netothr nettotalnr nr validnr nr expnr netnr nettotalnontendoioilivenet specnet specallchangechange     although a rare bird industry selling is a feature which occasionally flutters into view even on cocoa’s wilder moors – but bestto have the high strength binoculars at the ready though as it does not tend to indulge its hides of twitchers for too long even rarer –in fact ornithologically speaking a brandnew species – the market was treated to the sight of origin buying today as a bloomberg story winged its way across the newswires reporting the possibility of an ivorian government purchase of  tons not ‘tonnes’ according to the text from farmers and cooperatives who are still long it is true that when the wind blows even a turkey can fly and maybe  they 

In [102]:
from transformers import pipeline
summarizer = pipeline("summarization", model="Falconsai/text_summarization")

config.json: 100%|██████████████████████████████████████████████████████████| 1.49k/1.49k [00:00<00:00, 2.73MB/s]
model.safetensors: 100%|██████████████████████████████████████████████████████| 242M/242M [00:24<00:00, 9.89MB/s]
generation_config.json: 100%|████████████████████████████████████████████████████| 112/112 [00:00<00:00, 306kB/s]
tokenizer_config.json: 100%|████████████████████████████████████████████████| 2.32k/2.32k [00:00<00:00, 5.84MB/s]
spiece.model: 100%|███████████████████████████████████████████████████████████| 792k/792k [00:00<00:00, 8.18MB/s]
tokenizer.json: 100%|███████████████████████████████████████████████████████| 2.42M/2.42M [00:00<00:00, 3.42MB/s]
special_tokens_map.json: 100%|██████████████████████████████████████████████| 2.20k/2.20k [00:00<00:00, 9.01MB/s]


In [103]:
print(summarizer(comment, max_length=230, min_length=30, do_sample=False))

Token indices sequence length is longer than the specified maximum sequence length for this model (996 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'the market was treated to the sight of origin buying today as a bloomberg story winged its way across the newswires reporting the possibility of an ivorian government purchase of tons not ‘tonnes’ according to the text from farmers and cooperatives who are still long . there could be a few to those ‘familiar with the matter’ might be their intentions for the other tons allegedly dotted around the place or where that falls within the plans to ‘roll’ more than tons of unsold main crop into the'}]


In [123]:
output_string = StringIO()
with open('raw_data/pdf_reports/210113.pdf', 'rb') as fin:
    extract_text_to_fp(fin, output_string, laparams=LAParams(),
                       output_type='text', codec=None, maxpages=1)
output_string.getvalue()

FileNotFoundError: [Errno 2] No such file or directory: 'raw_data/pdf_reports/210113.pdf'

In [15]:
# Create a function to extract text
def text_extraction(element):
    # Extracting the text from the in-line text element
    line_text = element.get_text()
    
    # Find the formats of the text
    # Initialize the list with all the formats that appeared in the line of text
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Iterating through each character in the line of text
            for character in text_line:
                if isinstance(character, LTChar):
                    # Append the font name of the character
                    line_formats.append(character.fontname)
                    # Append the font size of the character
                    line_formats.append(character.size)
    # Find the unique font sizes and names in the line
    format_per_line = list(set(line_formats))
    
    # Return a tuple with the text in each line along with its format
    return (line_text, format_per_line)

In [22]:
# Extracting tables from the page

def extract_table(pdf_path, page_num, table_num):
    # Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    # Find the examined page
    table_page = pdf.pages[page_num]
    # Extract the appropriate table
    table = table_page.extract_tables()[table_num]
    return table

# Convert table into the appropriate format
def table_converter(table):
    table_string = ''
    # Iterate through each row of the table
    for row_num in range(len(table)):
        row = table[row_num]
        # Remove the line breaker from the wrapped texts
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Convert the table into a string 
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Removing the last line break
    table_string = table_string[:-1]
    return table_string

In [23]:
# Find the PDF path
pdf_path = 'raw_data/pdf_reports/210113.pdf'
# create a PDF file object
pdfFileObj = open(pdf_path, 'rb')
# create a PDF reader object
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Create the dictionary to extract text from each image
text_per_page = {}

# We extract the pages from the PDF
for pagenum, page in enumerate(extract_pages(pdf_path)):
    #Initialize the variables needed for the text extraction from the page
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []  
    text_from_tables = []
    page_content = []
    # Initialize the number of the examined tables
    table_num = 0
    first_element= True
    table_extraction_flag= False
    #Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    #Find the examined page
    page_tables = pdf.pages[pagenum]
    # Find the number of tables on the page
    tables = page_tables.find_tables()
    
    # Find all the elements
    page_elements = [(element.y1, element) for element in page._objs]
    # Sort all the elements as they appear in the page 
    page_elements.sort(key=lambda a: a[0], reverse=True)
    # Find the elements that composed a page
    for i,component in enumerate(page_elements):
        # Extract the position of the top side of the element in the PDF
        pos= component[0]
        # Extract the element of the page layout
        element = component[1]
        
        # Check if the element is a text element
        if isinstance(element, LTTextContainer):
            # Check if the text appeared in a table
            if table_extraction_flag == False:
                # Use the function to extract the text and format for each text element
                (line_text, format_per_line) = text_extraction(element)
                # Append the text of each line to the page text
                page_text.append(line_text)
                # Append the format for each line containing text
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Omit the text that appeared in a table
                pass

           # Check the elements for tables
        if isinstance(element, LTRect):
            # If the first rectangular element
            if first_element == True and (table_num+1) <= len(tables):
                # Find the bounding box of the table
                lower_side = page.bbox[3] - tables[table_num].bbox[3]
                upper_side = element.y1 
                # Extract the information from the table
                table = extract_table(pdf_path, pagenum, table_num)
                # Convert the table information in structured string format
                table_string = table_converter(table)
                # Append the table string into a list
                text_from_tables.append(table_string)
                page_content.append(table_string)
                # Set the flag as True to avoid the content again
                table_extraction_flag = True
                # Make it another element
                first_element = False
                # Add a placeholder in the text and format lists
                page_text.append('table')
                line_format.append('table')

In [39]:
tables

In [46]:
page_elements[1][1]

<LTRect 16.080,685.920,596.640,687.120>